1. Run neat on training data
2. Run neat on test data
3. TPOT - Create model and score model
4. Run neat on all new data then feed into TPOT recommended scikit learn model

In [114]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.utils import resample
from math import ceil

class Neat:
    
    def __init__(self, df, targetY, indexColumns=[], skipColumns=[]):        
        self.df = df        
        self.targetY = self._cleanColumnName(targetY)        
        self.indexColumns = self._cleanColumnNamesArray(indexColumns)     
        self.skipColumns = self._cleanColumnNamesArray(skipColumns)
        self.newData = None
        self.targetYMappings = {}
        self.numberColumns = []
        self.categoryColumns = []
        self.datetimeColumns = []
        self.medians = []
        self.lowerBounds = []
        self.upperBounds = []
        self.uniqueCategoryValues = {}
        self.valuesThatDontMapTo_Other = {}
        self.categoryFrequencies = {}
        self.targetYFrequencies = {}
        self.targetYUpsamplesNeeded = {}
        self.finalColumnNames = []
        self.columnsDropped = []
        # TargetY
        self._setTargetYMappings()
        self._convertTargetYToNumeric()
        self._dropNATargetYRows()        
        # Column Metadata
        self._cleanColumnNamesDF()        
        self._setColumnDataTypes()        
        # Index        
        self._dropDuplicatesAndMissingRowsIfIndexIsSpecified()    
        # Datetimes
        self._convertDatetimeToNumber()
        # Numbers
        self._saveMediansAndBounds()
        self._fixMissingNumValuesAndInfinity()
        self._fixHighLeveragePoints()
        # Categories
        self._saveUniqueCategoryValues()   
        self._saveCategoryFrequenciesAndValuesThatDontMapTo_Other()
        self._dropCategoryColumnsWithAllMissingValues()
        self._fixMissingCategoryValuesAndMapValuesTo_Other()
        self._applyOneHotEncoding()
        # Class Imbalance
        self._saveTargetYFrequencies()
        self._saveTargetYUpsamplesNeeded()
        self._fixTargetYImbalance()
        # Index        
        self._addIndex()
        # Get Final Column Names
        self._saveFinalColumnNames()
        
    def cleanNewData(self, newData):
        self.df = newData
        # Column Metadata
        self._cleanColumnNamesDF()                
        # Datetimes
        self._convertDatetimeToNumber()
        # Numbers
        self._fixMissingNumValuesAndInfinity()
        self._fixHighLeveragePoints()
        # Categories 
        self._fixMissingCategoryValuesAndMapValuesTo_Other()
        self._applyOneHotEncoding()
        # Index        
        self._addIndex() 
        # New Data
        self._newDataDropDroppedColumns()
        self._newDataAddMissingFinalColumnNames()
        self._newDataDropExtraColumnNames()
    
    def _cleanColumnNamesArray(self, columns):
        if type(columns) == str:
            columns = [columns]
        arr = []
        for column in columns:
            arr.append(self._cleanColumnName(column))
        return arr
    
    def _cleanColumnName(self, string):
        return string.strip().lower().replace(' ', '_')
    
    ########## TargetY ##########
    
    def _setTargetYMappings(self):
        if self.df[self.targetY].dtype == 'object': # a string
            i = 0      
            for value in self.df[self.targetY].unique():
                if value != None and value.strip() != "":
                    self.targetYMappings[value] = i
                    i = i + 1

    def _convertTargetYToNumeric(self):
        if self.df[self.targetY].dtype == 'object': # a string        
            self.df[self.targetY] = self.df[self.targetY].map(self.targetYMappings)
    
    def _dropNATargetYRows(self):    
        rowsToDrop = []
        for i, row in self.df.iterrows():    
            rowsToDrop.append(i) if np.isnan(row[self.targetY]) else None            
        self.df = self.df.drop(self.df.index[rowsToDrop])    
    
    ########## Column Metadata ##########
    
    def _cleanColumnNamesDF(self):
        self.df.columns = self.df.columns.str.strip().str.lower().str.replace(' ', '_')    
        
    def _setColumnDataTypes(self):
        columns = self.df.columns.values.tolist()
        for column in columns:    
            if column == self.targetY or column in indexColumns or column in skipColumns:
                continue
            elif self.df[column].dtype == 'int64' or self.df[column].dtype == 'float64':
                self.numberColumns.append(column)
            elif self.df[column].dtype == 'object':
                self.categoryColumns.append(column)
            else:
                self.datetimeColumns.append(column)  
                self.numberColumns.append(column)
    
    ########## Index ##########
    
    def _dropDuplicatesAndMissingRowsIfIndexIsSpecified(self):
        rowsToDrop = []
        if self.indexColumns != []:
            self.df = self.df.drop_duplicates(subset=self.indexColumns)
            for i, row in self.df.iterrows(): 
                for column in self.indexColumns:
                    if ((self.df[column].dtype == 'int64' or self.df[column].dtype == 'float64') and (np.isnan(row[column]) or np.isinf(row[column]))) or row[column] == None:
                        rowsToDrop.append(i)
        self.df = self.df.drop(self.df.index[rowsToDrop])    
     
    ########## Datetimes ##########
    
    def _convertDatetimeToNumber(self):        
        for column in self.datetimeColumns:
            values = []
            for i, row in self.df.iterrows(): 
                values.append((pd.datetime.now() - row[column]).days)
            self.df[column] = values  
    
    ########## Numbers ##########
        
    def _saveMediansAndBounds(self):        
        firstQuantiles = self.df.quantile(.25)
        thirdQuantiles = self.df.quantile(.75)
        
        self.medians = self.df.quantile(.50)
        self.lowerBounds = {}
        self.upperBounds = {}
        for column in self.numberColumns:            
            self.lowerBounds[column] = self.medians[column] - 2*(self.medians[column] - firstQuantiles[column])
            self.upperBounds[column] = self.medians[column] + 2*(thirdQuantiles[column] - self.medians[column])        
        
    def _fixMissingNumValuesAndInfinity(self):
        self.df = self.df.fillna(self.medians) # optionally: replace self.medians with 0
        self.df.replace([np.inf, -np.inf], np.nan)
        self.df = self.df.fillna(self.upperBounds)        
        
    def _fixHighLeveragePoints(self):
        for i, row in self.df.iterrows(): 
            for column in self.numberColumns:
                if row[column] > self.upperBounds[column]:
                    self.df.at[i, column] = self.upperBounds[column]
                if row[column] < self.lowerBounds[column]:
                    self.df.at[i, column] = self.lowerBounds[column]
      
    ########## Categories ##########
    
    def _saveUniqueCategoryValues(self):        
        for column in self.categoryColumns:
            self.uniqueCategoryValues[column] = []
            for value in self.df[column].unique():
                if value == None:
                    continue    
                self.uniqueCategoryValues[column].append(value)
            self.uniqueCategoryValues[column].append('_Other')            
            
    def _saveCategoryFrequenciesAndValuesThatDontMapTo_Other(self):
        for column in self.categoryColumns: 
            _otherFrequency = 0
            self.valuesThatDontMapTo_Other[column] = ['_Other']
            frequencyPercentage = pd.value_counts(self.df[column].values, sort=False, normalize=True)
            self.categoryFrequencies[column] = {}
            for value in self.uniqueCategoryValues[column]:                 
                if value == '_Other':
                    continue
                elif frequencyPercentage[value] < .05:                      
                    _otherFrequency = _otherFrequency + frequencyPercentage[value]
                else:
                    self.valuesThatDontMapTo_Other[column].append(value)
                    self.categoryFrequencies[column][value] = frequencyPercentage[value]            
            self.categoryFrequencies[column]['_Other'] = _otherFrequency
                    
    def _dropCategoryColumnsWithAllMissingValues(self):
        columnsToRemove = []
        for column in self.categoryColumns:
            if len(self.uniqueCategoryValues[column]) == 1 and self.uniqueCategoryValues[column][0] == '_Other':
                columnsToRemove.append(column)
                self.categoryColumns.remove(column)
                self.columnsDropped.append(column)
        self.df = self.df.drop(columnsToRemove, 1)         
            
            
    def _fixMissingCategoryValuesAndMapValuesTo_Other(self):
        for i, row in self.df.iterrows(): 
            for column in self.categoryColumns:        
                if row[column] == None:
                    self.df.at[i, column] = self._getRandomCategoryBasedOnFrequencies(column)
                elif row[column] not in self.valuesThatDontMapTo_Other[column]:
                    self.df.at[i, column] = '_Other'
            
    def _getRandomCategoryBasedOnFrequencies(self, column):
        chosenValue, prevValue, cumulativeProbability = None, None, 0
        randomNumber = np.random.uniform(0,1,1)[0]
        for value in self.uniqueCategoryValues[column]:
            if value in self.valuesThatDontMapTo_Other[column]:
                probabilityOfValue, prevValue = self.categoryFrequencies[column][value], value
                cumulativeProbability = cumulativeProbability + probabilityOfValue
                if cumulativeProbability > randomNumber:
                    chosenValue = value
                    break
        return prevValue if chosenValue == None else chosenValue            

    def _applyOneHotEncoding(self): # don't drop_first => one hot encoding instead of dummy encoding
        for column in self.categoryColumns:
            self.df = pd.concat([self.df.drop(column, axis=1), pd.get_dummies(self.df[column], prefix=column+"_", drop_first=False)], axis=1)            

    ########## Class Imbalance ##########
        
    def _saveTargetYFrequencies(self):           
        self.targetYFrequencies = pd.value_counts(self.df[self.targetY].values, sort=True, normalize=False)        
        
    def _saveTargetYUpsamplesNeeded(self): 
        maxValue = None
        for value in self.targetYMappings.values():
            frequency = self.targetYFrequencies[value]
            if maxValue == None or frequency > maxValue:
                maxValue = frequency
        
        minValue = ceil(maxValue / 2)    

        for value in self.targetYMappings.values():
            actualFrequency = self.targetYFrequencies[value]
            idealTargetYFrequency = minValue if actualFrequency < minValue else actualFrequency
            self.targetYUpsamplesNeeded[value] = idealTargetYFrequency - actualFrequency        
        
    def _fixTargetYImbalance(self):    
        for value in self.targetYMappings.values():
            samplesToGet = self.targetYUpsamplesNeeded[value]
            if samplesToGet > 0:
                upsampleRows = resample(self.df[self.df[self.targetY]==value],
                                    replace=True,
                                    n_samples=samplesToGet,
                                    random_state=123)
                self.df = pd.concat([self.df, upsampleRows])
                
    ########## Index ##########
        
    def _addIndex(self): 
        indexColumns = []
        self.df['_id'] = np.arange(1,len(self.df.index)+1)
        if self.indexColumns != []:
            indexColumns = list(self.indexColumns)
        indexColumns.append('_id')   
        self.df = self.df.set_index(indexColumns)        
    
    ########## Get Final Column Names ##########
    
    def _saveFinalColumnNames(self):
        self.finalColumnNames = list(self.df)
        
    ########## New Data ##########        
        
    def _newDataDropDroppedColumns(self):
        self.df = self.df.drop(self.columnsDropped, axis=1)
        
    def _newDataAddMissingFinalColumnNames(self):
        # Assuming only category columns will be missing
        for column in self.finalColumnNames:
            if column not in list(self.df):
                self.df[column] = np.zeros((len(self.df.index),1))
                
    def _newDataDropExtraColumnNames(self): # This hopefully does nothing - using it anyway
        columnsToDrop = []
        for column in list(self.df):
            if column not in self.finalColumnNames:
                columnsToDrop.append(column)
        self.df = self.df.drop(columnsToDrop, axis=1)

                
                
                
        
df = pd.DataFrame({'col1': ['a','b','c','a','a','g','b','a','i','t','a','b','c','a','a','g','b','a','i','t','b','a','i','t','a','b','c','a','a','g','b','a','i','t','a','b','c','a','a','g','b','a','i','t','b','a','i','t']
                  , 'col2': [None,None,None,9,5,10,11,12,13,14,None,None,None,9,5,10,11,12,13,14,11,12,13,14,None,None,None,9,5,10,11,12,13,14,None,None,None,9,5,10,11,12,13,14,11,12,13,14]
                  , 'col3': ['test1','test1','test1','test3',None,None,'test1','test1','test2','test2','test1','test1','test1','test1',None,None,'test1','test1','test2','test2', 'test1','test1','test2','test2','test1','test1','test1','test1',None,None,'test1','test1','test2','test2','test1','test1','test1','test1',None,None,'test1','test1','test2','test2', 'test1','test1','test2','test2']
                  , 'col4': [None, 5, 3 ,6 ,8, 9, 14, 87, 999 ,9999,None, 5, 3 ,6 ,8, 9, 14, 87, 999 ,9999, 14, 87, 999 ,9999,None, 5, 3 ,6 ,8, 9, 14, 87, 999 ,9999,None, 5, 3 ,6 ,8, 9, 14, 87, 999 ,9999, 14, 87, 999 ,9999]
                  , 'col5': ['a','a',None,None,'adsf','bas',None,None,None,None,None,None,None,None,'adsf','bas',None,None,None,None,None,None,None,None,'a','a',None,None,'adsf','bas',None,None,None,None,None,None,None,None,'adsf','bas',None,None,None,None,None,None,None,None]})                

# df2 = pd.DataFrame({'year': [2015, 2016, 2016, 2016, 2016, 2017, 2017, 2017, 2017, None],
#                        'month': [2, 3, 3, 3, 3, 3, 3, 3, 1, None],
#                        'day': [4, 5, 5, 5, 5, 5, 5, 5, 20, None]})
# datetime = pd.to_datetime(df2)
# df2['datetime'] = datetime
# df2 = df2.drop(['year','month','day'], axis=1)

# datetimeColumns = ['datetime','datetime2']

# df['col6'] = df2['datetime']

targetY = 'col1'
#indexColumns = 'col2'
indexColumns = ['col4']
indexColumns = []


neat = Neat(df, targetY, indexColumns)
    
print(neat.df)
#df = neat.df

neat.cleanNewData(df)

neat.df


column col3 value test1 0.675
column col3 value test2 0.3
column col5 value a 0.333333333333
column col5 value adsf 0.333333333333
column col5 value bas 0.333333333333
     col1  col2    col4  col3___Other  col3__test1  col3__test2  col5__a  \
_id                                                                        
1       0  11.5    14.0             0            1            0        1   
2       1  11.5     5.0             0            1            0        1   
3       2  11.5     3.0             0            1            0        1   
4       0   9.0     6.0             1            0            0        0   
5       0   8.5     8.0             0            1            0        0   
6       3  10.0     9.0             0            1            0        0   
7       1  11.0    14.0             0            1            0        1   
8       0  12.0    87.0             0            1            0        0   
9       4  13.0   999.0             0            0            1        0

col1  col2    col4  col3___Other  col3__test1  col3__test2  col5__a  \
_id                                                                        
1       0  11.5    14.0             0            1            0        1   
2       1  11.5     5.0             0            1            0        1   
3       2  11.5     3.0             0            1            0        1   
4       0   9.0     6.0             1            0            0        1   
5       0   8.5     8.0             1            0            0        0   
6       3  10.0     9.0             0            1            0        0   
7       1  11.0    14.0             0            1            0        1   
8       0  12.0    87.0             0            1            0        0   
9       4  13.0   999.0             0            0            1        0   
10      5  14.0  1984.0             0            0            1        1   
11      0  11.5    14.0             0            1            0        0   
12      1  11.5     5.0             0            1            0        1   
13      2  11.5     3.0             0            1            0        0   
14      0   9.0     6.0             0            1            0        1   
15      0   8.5     8.0             0            1            0        0   
16      3  10.0     9.0             0            1            0        0   
17      1  11.0    14.0             0            1            0        0   
18      0  12.0    87.0             0            1            0        0   
19      4  13.0   999.0             0            0            1        0   
20      5  14.0  1984.0             0            0            1        1   
21      1  11.0    14.0             0            1            0        0   
22      0  12.0    87.0             0            1            0        0   
23      4  13.0   999.0             0            0            1        0   
24      5  14.0  1984.0             0            0            1        0   
25      0  11.5    14.0             0            1            0        1   
26      1  11.5     5.0             0            1            0        1   
27      2  11.5     3.0             0            1            0        1   
28      0   9.0     6.0             0            1            0        0   
29      0   8.5     8.0             0            1            0        0   
30      3  10.0     9.0             0            1            0        0   
31      1  11.0    14.0             0            1            0        1   
32      0  12.0    87.0             0            1            0        0   
33      4  13.0   999.0             0            0            1        0   
34      5  14.0  1984.0             0            0            1        0   
35      0  11.5    14.0             0            1            0        0   
36      1  11.5     5.0             0            1            0        0   
37      2  11.5     3.0             0            1            0        1   
38      0   9.0     6.0             0            1            0        0   
39      0   8.5     8.0             0            0            1        0   
40      3  10.0     9.0             0            1            0        0   
41      1  11.0    14.0             0            1            0        0   
42      0  12.0    87.0             0            1            0        0   
43      4  13.0   999.0             0            0            1        0   
44      5  14.0  1984.0             0            0            1        0   
45      1  11.0    14.0             0            1            0        0   
46      0  12.0    87.0             0            1            0        0   
47      4  13.0   999.0             0            0            1        0   
48      5  14.0  1984.0             0            0            1        1   

     col5__adsf  col5__bas  
_id                         
1             0          0  
2             0          0  
3             0          0  
4             0          0  
5             1          0  
6